In [180]:
import pandas as pd

pd.options.display.max_rows = 5
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 20

In [181]:
horse_vs_df = pd.read_csv('3.4.horse_vs_data.csv')
horse_vs_df = horse_vs_df.iloc[:,:28]
df = pd.read_csv('2.2.add.日付_df.csv')
print(len(horse_vs_df))
print(len(df))

15883
1366


In [182]:
horse_vs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15883 entries, 2018105875 to 2017104925
Data columns (total 28 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   日付        15883 non-null  object 
 1   開催        15868 non-null  object 
 2   天気        15864 non-null  object 
 3   R         15880 non-null  float64
 4   レース名      15883 non-null  object 
 5   映像        0 non-null      float64
 6   頭数        15883 non-null  float64
 7   枠番        15880 non-null  float64
 8   馬番        15883 non-null  float64
 9   オッズ       15775 non-null  float64
 10  人気        15777 non-null  float64
 11  着順        15840 non-null  object 
 12  騎手        15883 non-null  object 
 13  斤量        15883 non-null  float64
 14  距離        15883 non-null  object 
 15  馬場        15867 non-null  object 
 16  馬場指数      15752 non-null  object 
 17  タイム       15740 non-null  object 
 18  着差        15739 non-null  float64
 19  ﾀｲﾑ指数     15752 non-null  object 
 20  通過        1576

In [190]:
pd.options.display.max_rows = None
print(df.info())
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1366 entries, 0 to 1365
Data columns (total 17 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   着順       1366 non-null   object 
 1   枠        1366 non-null   int64  
 2   馬番       1366 non-null   int64  
 3   馬名       1366 non-null   object 
 4   性齢       1366 non-null   object 
 5   斤量       1366 non-null   float64
 6   騎手       1366 non-null   object 
 7   タイム      1357 non-null   object 
 8   着差       1249 non-null   object 
 9   人気       1366 non-null   object 
 10  単勝オッズ    1360 non-null   float64
 11  後3F      1357 non-null   float64
 12  コーナー通過順  1360 non-null   object 
 13  厩舎       1366 non-null   object 
 14  馬体重(増減)  1363 non-null   object 
 15  性        1366 non-null   object 
 16  齢        1366 non-null   int64  
dtypes: float64(3), int64(3), object(11)
memory usage: 181.5+ KB
None


着順           0
枠            0
馬番           0
馬名           0
性齢           0
斤量           0
騎手           0
タイム          9
着差         117
人気           0
単勝オッズ        6
後3F          9
コーナー通過順      6
厩舎           0
馬体重(増減)      3
性            0
齢            0
dtype: int64

In [184]:
# df[df["後3F"].isna()]
df["人気"].value_counts()

2.0     108
5.0     108
4.0     108
1.0     108
3.0     108
6.0     108
7.0     106
8.0     105
9.0     101
10.0     96
11.0     88
12.0     81
13.0     56
14.0     44
15.0     20
16.0     15
Name: 人気, dtype: int64

In [185]:
# pd.options.display.max_rows = None
# for i in df.columns:
#   print(df[i].value_counts())

## メインデータの前処理

In [186]:
def preprocessing(df):
  # 性齢
  df["性"] = df["性齢"].map(lambda x:str(x)[0])
  df["齢"] = df["性齢"].map(lambda x:str(x)[1:]).astype(int)
  # 人気
  df.loc[df['人気'] == "nan", '人気'] = 99
  df["人気"] = df["人気"].map(lambda x:str(x).replace('.0', ''))
  df["人気"] = df["人気"].astype(int)
  # タイム
  df.loc[df['タイム'] == "-", 'タイム'] = "9:99.9"
  # 着差
  df.loc[df['着差'] == "-", '着差'] = "除外"
  df["着差"] = df["着差"].fillna("１着")
  # オッズ
  df.loc[df['単勝オッズ'] == "-", '単勝オッズ'] = "0.0"
  df["単勝オッズ"] = df["単勝オッズ"].astype(float)
  # 後3F
  df.loc[df['後3F'] == "-", '後3F'] = "9.9"
  df["後3F"] = df["後3F"].astype(float)
  # コーナー通過順
  df.loc[df['コーナー通過順'] == "-", 'コーナー通過順'] = "0-0-0-0"
  # 馬体重
  # df = df[~(df["馬体重(増減)"].astype(str).str.contains("\."))]
  df["体重"] = df["馬体重(増減)"].str.split('(',expand=True)[0].astype(int)
  df["体重変化"] = df["馬体重(増減)"].str.split('(',expand=True)[1].str[:-1].astype(int)
  df.drop(["馬体重(増減)",'性齢'],axis=1, inplace=True)
  # 日付
  df["date"] = pd.to_datetime(df["date"],format="%Y年%m月%d日")

  return df  

## HorseResultsクラス

In [187]:
class HorseResults:
  def __init__(self,horse_vs_df):
    self.horse_results = horse_vs_df[["日付","着順","賞金"]]
    
  def preprocessing(self):
    df = self.horse_results.copy()
    # 着順
    df["着順"] = df["着順"].map(lambda x:str(x).replace('.0', ''))
    # 日付
    df["date"] = pd.to_datetime(df["日付"])
    df.drop(["日付"],axis=1, inplace=True)
    
    self.horse_results = df


In [188]:
hr = HorseResults(horse_vs_df)
hr.preprocessing()
hr.horse_results
hr.horse_results.index = hr.horse_results.index.map(lambda x:str(x))

In [189]:
results_p = preprocessing(df)
len(results_p)

ValueError: invalid literal for int() with base 10: 'nan'

In [ ]:
sample_data = results_p["date"][0]
sample_data

sample_dataの日付に走った馬のリスト


In [ ]:
pd.options.display.max_rows = None

horse_id_list = results_p[results_p["date"] == sample_data]["horse_id"]
horse_id_list

In [ ]:
hr.horse_results.loc[horse_id_list]

In [ ]:
hr.horse_results

In [ ]:
horse_vs_df["2018104222"]